In this notebook, we are going to go through the process of understanding fully the code that is being presented to us. 

In [23]:
import itertools
import numpy as np
import cvxpy as cp
import pandas as pd
from enum import Enum
from typing import TypeVar
from pydantic import BaseModel


CENTER_NODE = "CENTAURI"  # Name of center Node
ORIGIN_TOKEN = "WETH"
OBJ_TOKEN = "ATOM"

MAX_RESERVE = 1e10

TAssetId = TypeVar("TAssetId")
TNetworkId = TypeVar("TNetworkId")

def populate_chain_dict(chains: dict[TNetworkId, list[TAssetId]], center_node: TNetworkId):
    # Add tokens with denom to Center Node
    # Basic IBC transfer
    for chain, tokens in chains.items():
        if chain != center_node:
            chains[center_node].extend(f"{chain}/{token}" for token in tokens)

    1# Add tokens from Center Node to outers
    
    # Simulate IBC transfer through Composable Cosmos
    for chain, tokens in chains.items():
        if chain != center_node:
            chains[chain].extend(
                f"{center_node}/{token}"
                for token in chains[center_node]
                if f"{chain}/" not in token
            )

# This function takes in your tokens, market makers, and reserves and returns the optimal trade. 
# You get the deltas, lambdas, psi, and etas           
def solve(
    all_tokens: list[TAssetId],
    all_cfmms: list[tuple[TAssetId, TAssetId]],
    reserves: list[np.ndarray[np.float64]],
    cfmm_tx_cost: list[float],
    fees: list[float],
    ibc_pools: int,
    origin_token: TAssetId,
    number_of_init_tokens: float,
    obj_token: TAssetId,
    force_eta: list[float] = None,
):
    
    # Build local-global matrices
    count_tokens = len(all_tokens)
    count_cfmms = len(all_cfmms)

    current_assets = np.zeros(count_tokens)
    current_assets[all_tokens.index(origin_token)] = number_of_init_tokens

    A = []
    for cfmm in all_cfmms:
        n_i = len(cfmm)
        A_i = np.zeros((count_tokens, n_i))
        for i, token in enumerate(cfmm):
            A_i[all_tokens.index(token), i] = 1
        A.append(A_i)

    # Build variables
    
    # tendered (given) amount
    deltas = [cp.Variable(len(l), nonneg=True) for l in all_cfmms]
    
    # received (wanted) amounts
    lambdas = [cp.Variable(len(l), nonneg=True) for l in all_cfmms]
    eta = cp.Variable(
        count_cfmms, nonneg=True
    )  # Binary value, indicates tx or not for given pool

    # network trade vector - net amount received over all trades(transfers/exchanges)
    psi = cp.sum([A_i @ (LAMBDA - DELTA) for A_i, DELTA, LAMBDA in zip(A, deltas, lambdas)])
    
    # Objective is to trade number_of_init_tokens of asset origin_token for a maximum amount of asset objective_token
    obj = cp.Maximize(psi[all_tokens.index(obj_token)] - eta @ cfmm_tx_cost)

    # Reserves after trade
    new_reserves = [
        R + gamma_i * D - L for R, gamma_i, D, L in zip(reserves, fees, deltas, lambdas)
    ]

    # Trading function constraints
    constrains = [
        psi + current_assets >= 0,
    ]

    # Pool constraint (Uniswap v2 like)
    for i in range(count_cfmms - ibc_pools):
        constrains.append(cp.geo_mean(new_reserves[i]) >= cp.geo_mean(reserves[i]))

    # Pool constraint for IBC transfer (constant sum)
    # NOTE: Ibc pools are at the bottom of the cfmm list
    for i in range(count_cfmms - ibc_pools, count_cfmms):
        constrains.append(cp.sum(new_reserves[i]) >= cp.sum(reserves[i]))
        constrains.append(new_reserves[i] >= 0)

    # Enforce deltas depending on pass or not pass variable
    # MAX_RESERVE should be big enough so delta <<< MAX_RESERVE
    for i in range(count_cfmms):
        constrains.append(deltas[i] <= eta[i] * MAX_RESERVE)
        if force_eta:
            constrains.append(eta[i] == force_eta[i])

    # Set up and solve problem
    prob = cp.Problem(obj, constrains)
    prob.solve(verbose= True)

    print(
        f"\033[1;91mTotal amount out: {psi.value[all_tokens.index(obj_token)]}\033[0m"
    )

    for i in range(count_cfmms):
        print(
            f"Market {all_cfmms[i][0]}<->{all_cfmms[i][1]}, delta: {deltas[i].value}, lambda: {lambdas[i].value}, eta: {eta[i].value}",
        )
    
    # deltas[i] - how much one gives to pool i
    # lambdas[i] - how much one wants to get from pool i
    return deltas, lambdas, psi, eta




### Environment set up

In this next section of code, we establish what the center node is and the set of constant function market makers that exist. There are two types of them: 
* Regular constant function market makers which help you to exchange one token for another 
* IBC pools which are constant sum market makers which represent the movement of one token into another one

One thing that we could do is operationalize this using objects and stuff. Make it easier to track what is going on in the system. 

In [24]:
CENTER_NODE = "CENTAURI"  # Name of center Node

ORIGIN_TOKEN = "WETH"
OBJ_TOKEN = "ATOM"

chains: dict[str, list[str]] = {
    "ETHEREUM": ["WETH", "USDC", "SHIBA"],
    CENTER_NODE: [],
    "OSMOSIS": ["ATOM","SCRT"],
}

populate_chain_dict(chains,CENTER_NODE)

all_tokens = []
all_cfmms = []
reserves = []
fees = []
cfmm_tx_cost = []
ibc_pools = 0
tol = 1e-4

In [25]:
for other_chain, other_tokens in chains.items():
    
    # We add the tokens to the list of tokens. These are tokens that exist on that chain
    all_tokens.extend(other_tokens)
    
    # We also add all market makers to the list of market makers using a combination of 2 tokens
    all_cfmms.extend(itertools.combinations(other_tokens, 2))
    
for cfmm in all_cfmms:
    
    # Adding a simulation of reserves and transaction costs for each
    reserves.append(np.random.uniform(9500, 10051, 2))
    cfmm_tx_cost.append(np.random.uniform(0, 20))


In [26]:
# Creating the IBC pools here which represent constant sum pools 
# These are not really "pools" but the bridges that you have to go through to get to the other chain
for token_on_center in chains[CENTER_NODE]:
        for other_chain, other_tokens in chains.items():
            if other_chain != CENTER_NODE:
                for other_token in other_tokens:
                    # Check wether the chain has the token in center, or the other way around
                    # Could cause problems if chainName == tokensName (for example OSMOSIS)
                    if other_token in token_on_center or token_on_center in other_token:
                        all_cfmms.append((token_on_center, other_token))
                        reserves.append(np.random.uniform(10000, 11000, 2))
                        cfmm_tx_cost.append(np.random.uniform(0, 20))
                        ibc_pools += 1
                        
# simulate random fees for all the CFMMs, including those that are IBC pools
fees.extend(np.random.uniform(0.97, 0.999) for _ in range(len(all_cfmms)))

### Solving the problem

We now are going to solve the optimal routing problem using python's cvxpy module. 

In [106]:
input_amount = 2000
deltas, lambdas, psi, n = solve(
    all_tokens,
    all_cfmms,
    reserves,
    cfmm_tx_cost,
    fees,
    ibc_pools,
    ORIGIN_TOKEN,
    input_amount,
    OBJ_TOKEN,
)

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 11 02:18:15 PM: Your problem has 200 variables, 91 constraints, and 0 parameters.
(CVXPY) Jan 11 02:18:15 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 11 02:18:15 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 11 02:18:15 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 11 02:18:15 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 11 02:18:15 PM: Compiling problem (target solver=ECOS).
(C

(CVXPY) Jan 11 02:18:15 PM: Applying reduction Dcp2Cone
(CVXPY) Jan 11 02:18:15 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jan 11 02:18:15 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Jan 11 02:18:15 PM: Applying reduction ECOS


/home/stephonh/.local/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


(CVXPY) Jan 11 02:18:15 PM: Finished problem compilation (took 2.342e-01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jan 11 02:18:15 PM: Invoking solver ECOS  to obtain a solution.

ECOS 2.0.10 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -2.091e+03  -1.020e+05  +4e+06  4e-01  1e+00  1e+00  1e+04    ---    ---    1  1  - |  -  - 
 1  -8.580e+03  -3.694e+04  +1e+06  9e-02  5e-01  1e+02  3e+03  0.6852  2e-02   1  1  1 |  0  0
 2  -5.951e+03  -1.227e+04  +3e+05  2e-02  1e-01  3e+02  7e+02  0.8953  1e-01   1  1  1 |  0  0
 3  -2.875e+03  -4.099e+03  +5e+04  4e-03  2e-02  8e+01  1e+02  0.8385  3e-02   1  1  1 |  0  0
 4  -9.764e+02  -1.196e+03  +1e+04

/home/stephonh/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [107]:
to_look_n: list[float] = []
for i in range(len(all_cfmms)):
    to_look_n.append(n[i].value)

_max = 0
for t in sorted(to_look_n):
    try:
        d2, l2, p2, n2 =  solve(
            all_tokens,
            all_cfmms,
            reserves,
            cfmm_tx_cost,
            fees,
            ibc_pools,
            ORIGIN_TOKEN,
            input_amount,
            OBJ_TOKEN,
            [1 if value <= t else 0 for value in to_look_n],
        )
        if psi.value[all_tokens.index(OBJ_TOKEN)] > _max:
            d_max, l_max, p_max, n_max = d2, l2, p2, n2 
        print("---")
    except:
        continue
eta = n_max
eta_change = True
    

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 11 02:18:37 PM: Your problem has 200 variables, 131 constraints, and 0 parameters.
(CVXPY) Jan 11 02:18:37 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 11 02:18:37 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 11 02:18:37 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 11 02:18:37 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 11 02:18:37 PM: Compiling problem (target solver=ECOS).
(

In [108]:
lastp_value = psi.value[all_tokens.index(OBJ_TOKEN)]
while eta_change:
    try:
        eta_change = False

        for idx, delta in enumerate(d_max):
            if all(delta_i.value < 1e-04 for delta_i in delta):
                n_max[idx] = 0
                eta_change = True
        d_max, l, psi, eta = solve(
            all_tokens,
            all_cfmms,
            reserves,
            cfmm_tx_cost,
            fees,
            ibc_pools,
            ORIGIN_TOKEN,
            input_amount,
            OBJ_TOKEN,
            eta,
        )

    except:
        continue

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 11 02:21:01 PM: Your problem has 240 variables, 131 constraints, and 0 parameters.
(CVXPY) Jan 11 02:21:01 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 11 02:21:01 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 11 02:21:01 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 11 02:21:01 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 11 02:21:01 PM: Compiling problem (target solver=ECOS).
(

In [109]:
deltas, lambdas, psi, eta = solve(
                    all_tokens,
                    all_cfmms,
                    reserves,
                    cfmm_tx_cost,
                    fees,
                    ibc_pools,
                    ORIGIN_TOKEN,
                    input_amount,
                    OBJ_TOKEN,
                    eta,
                )
m = len(all_cfmms)
for i in range(m):
    print(
        f"Market {all_cfmms[i][0]}<->{all_cfmms[i][1]}, delta: {deltas[i].value}, lambda: {lambdas[i].value}, eta: {eta[i].value}",
    )

print(psi.value[all_tokens.index(OBJ_TOKEN)],lastp_value)

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 11 02:22:15 PM: Your problem has 240 variables, 131 constraints, and 0 parameters.
(CVXPY) Jan 11 02:22:15 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 11 02:22:15 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 11 02:22:15 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 11 02:22:15 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 11 02:22:15 PM: Compiling problem (target solver=ECOS).
(

In [28]:
 # Build local-global matrices
count_tokens = len(all_tokens)
count_cfmms = len(all_cfmms)
A = []
for cfmm in all_cfmms:
    n_i = len(cfmm)
    A_i = np.zeros((count_tokens, n_i))
    for i, token in enumerate(cfmm):
        A_i[all_tokens.index(token), i] = 1
    A.append(A_i)

In [29]:
def get_swap_instructions(i: int): 
    psi_i = A[i] @ (lambdas[i].value - deltas[i].value)
    
    # Get the indices where the values are not 0
    indices = list(np.nonzero(psi_i)[0])
    
    first_token = all_tokens[indices[0]]
    second_token = all_tokens[indices[1]]
    
    # Get the amount of tokens that are being traded
    first_token_amount = psi_i[indices[0]]
    second_token_amount = psi_i[indices[1]]
    
    if first_token_amount < 0:
        return f"Trade {-first_token_amount} {first_token} for {second_token_amount} {second_token}"
    else:
        return f"Trade {-second_token_amount} {second_token} for {first_token_amount} {first_token}"

In [30]:
for i in range(len(all_cfmms)):
    print(get_swap_instructions(i))

Trade 199.57826386917714 WETH for 195.550559018189 USDC
Trade 251.6344785814913 WETH for 247.7142051626943 SHIBA
Trade 310.64743764587143 WETH for 299.14934626284384 CENTAURI/OSMOSIS/ATOM
Trade 234.08856742037057 WETH for 228.9466891786847 CENTAURI/OSMOSIS/SCRT
Trade 0.006397628887808525 USDC for 0.006368674402247055 SHIBA
Trade 195.54302679085006 USDC for 193.84669972205623 CENTAURI/OSMOSIS/ATOM
Trade 4.123920584735185e-05 USDC for 1.370918617685853e-05 CENTAURI/OSMOSIS/SCRT
Trade 131.25124555901945 SHIBA for 129.3198317585649 CENTAURI/OSMOSIS/ATOM
Trade 116.41882013976233 SHIBA for 117.21439989196746 CENTAURI/OSMOSIS/SCRT
Trade 4.655274403442046 CENTAURI/OSMOSIS/SCRT for 4.551066582593992 CENTAURI/OSMOSIS/ATOM
Trade 83.05812954888717 ETHEREUM/WETH for 80.54246832093544 ETHEREUM/USDC
Trade 0.14519088792304272 ETHEREUM/WETH for 0.1388394525566607 ETHEREUM/SHIBA
Trade 117.82807563128102 ETHEREUM/WETH for 109.04005691843027 OSMOSIS/ATOM
Trade 79.79808330986393 ETHEREUM/WETH for 77.211491

In [31]:
class Swap: 
    
    def __init__(self, token_in: str, token_out: str, amount_in: float, amount_out: float):
        self.token_in = token_in
        self.token_out = token_out
        self.amount_in = amount_in
        self.amount_out = amount_out
        
    def __str__(self) -> str:
        return f"Trade {self.amount_in} {self.token_in} for {self.amount_out} {self.token_out}"
    
    def __repr__(self) -> str:
        return f'Swap("{self.token_in}", "{self.token_out}", {self.amount_in}, {self.amount_out})'
    
    

In [42]:
def create_swap(i: int) -> Swap: 
    psi_i = A[i] @ (deltas[i].value - lambdas[i].value)
    
    # Get the indices where the values are not 0
    indices = list(np.nonzero(psi_i)[0])
    
    first_token = all_tokens[indices[0]]
    second_token = all_tokens[indices[1]]
    
    # Get the amount of tokens that are being traded
    first_token_amount = psi_i[indices[0]]
    second_token_amount = psi_i[indices[1]]
    
    if first_token_amount > 0:
        return Swap(first_token, second_token, first_token_amount, -second_token_amount)
    else:
        return Swap(second_token, first_token, second_token_amount, -first_token_amount)
    

In [81]:
swaps = [] 
for i in range(len(all_cfmms)):
    swaps.append(create_swap(i))

Now, we have all of these in object form, we can get swaps that are doing certain things. A simple algorithm that I'm thinking of is this: 
* Start with your start token (in this case WETH)
    * Find all swaps where we are starting with WETH
    * Execute those swaps and take them out of the list of swaps
    * Keep track of how much of the other tokens including the starting one that we have
    * For each of those tokens, find the swaps that start with each of those and repeat this process 
    * Keep going until you have no more tokens except the target token -> can probably do some sort of while loop to handle this or something. 

In [86]:
inventory = {token: 0 for token in all_tokens}
inventory[ORIGIN_TOKEN] = input_amount

inventories = []

In [87]:
swaps_copy = swaps.copy()
swaps_sorted = []

In [88]:
start_token = 'WETH'

# Get all swaps that start with the start_token
start_swaps = [swap for swap in swaps_copy if swap.token_in == start_token]

for s in start_swaps: 
    inventory[s.token_in] -= s.amount_in
    inventory[s.token_out] += s.amount_out
    
    # Remove the swap from the list of swaps in swaps_copy
    swaps_sorted.append(s)
    swaps_copy.remove(s)
    
inventories.append(inventory.copy())
    
while len(swaps_copy) > 0:
    for token, amount in inventory.items():
        token_swaps = [swap for swap in swaps_copy if swap.token_in == token]
        
        if len(token_swaps) == 0: 
            continue
        else: 
            for s in token_swaps:
                inventory[s.token_in] -= s.amount_in
                inventory[s.token_out] += s.amount_out
                
                # Remove the swap from the list of swaps in swaps_copy
                swaps_sorted.append(s)
                swaps_copy.remove(s)
    inventories.append(inventory.copy())

In [90]:
example_swap = swaps_sorted[0]

In [92]:
example_swap.token_out

'USDC'

In [93]:
next_swaps = [swap for swap in swaps_sorted if swap.token_in == example_swap.token_out]

In [96]:

next_swaps_2 = [swap for swap in swaps_sorted if swap.token_in == next_swaps[0].token_out]

In [98]:
next_swaps_3 = [swap for swap in swaps_sorted if swap.token_in == next_swaps_2[0].token_out]

In [100]:
next_swaps_4 = [swap for swap in swaps_sorted if swap.token_in == next_swaps_3[0].token_out]

In [102]:
next_swaps_5 = [swap for swap in swaps_sorted if swap.token_in == next_swaps_4[0].token_out]

In [ ]:
from typing import List

class Node:
    def __init__(self, token: str):
        self.next: List[Node] = []
